# Accessing Global Forecast System (GFS) data from Azure public container

### Product focus:
There are five products in GFS dataset:
- gfs: forecast data from the Global Forecast System (GFS)
- gdas: forecast data gridded with the Global Data Assimilation System (GDAS)
- gfsmos: model output statistics from the GFS MOS suite
- sst: sea surface temperature forecasts produced by the NCEP Sea Surface Temperature (SST) models
- enkfgdas: data assimilated using the GSI Hybrid/4DEnVar Data Assimilation system

**This notebook only focuses on `GFS - Global longitude-latitude grid` product, with most commonly used parameters.**
### This notebook does the following:
- Fetch GRIB data files from Azure URLs
- Read GRIB file into xarray dataset
- Convert Xarray dataset to pandas dataframe
- Create mulitiple visualizations including animations using the GFS data
### References:
- [NCEI Environmental Modeling Center/GFS](https://www.emc.ncep.noaa.gov/emc/pages/numerical_forecast_systems/gfs.php)
- [GFS Products Inventory](https://www.nco.ncep.noaa.gov/pmb/products/gfs/#GFS)
- [Azure Storage Resources](https://planetarycomputer.microsoft.com/dataset/storage/noaa-gfs)
- [Xarray Library Documentation](https://docs.xarray.dev/en/stable/index.html)

---

### Run the following cell to install the dependencies

In [ ]:
# !python -m pip install --upgrade pip
# %pip install xarray[complete]
# %pip install eccodes
# %pip install ecmwflibs
# %pip install cfgrib
# %pip install numpy==1.23.0
# %pip install alive_progress
# %pip install cartopy
# %pip install imageio

In [ ]:
# Run this to restart the notebook if you are in the Databricks environment

# dbutils.library.restartPython() 

---

#### Define a function `file_path()` to fetch the urls from a public Azure container

In [ ]:
import os

def file_path(cycle_runtime: int, forecast_hour: int, year: int, month: int, day: int, resolution_degree: float) -> str:
    prefix_path = "https://noaagfs.blob.core.windows.net/"
    product_name = "gfs"
    resolution_split = str(resolution_degree).split(".")
    file_path = (
        f"{product_name}/{product_name}.{year}{month:>02}{day:>02}/"
        f"{cycle_runtime:>02}/atmos/{product_name}.t{cycle_runtime:>02}z."
        f"pgrb2.{resolution_split[0]}p{resolution_split[1]:<02}.f{forecast_hour:>03}"
    )
    whole_path = os.path.join(prefix_path, file_path)

    return whole_path


#### Define a function `read_into_xarray_dataset()` to read given url into Xarray dataset

See `key_words.json` for filtering keywords reference

In [ ]:
import xarray as xr
import urllib.request
from urllib.error import HTTPError
from typing import Optional
import json

with open("key_words.json") as f:
    key_words = json.load(f)

key_details = key_words["key_details"]

def read_into_xarray_dataset(URL: str, level: str, step: Optional[str] = None):
    try:
        filename, _ = urllib.request.urlretrieve(URL)
        step_key = ["atmosphere", "surface", "lowCloudLayer", "middleCloudLayer", "highCloudLayer"]

        if level in step_key:
            ds = xr.open_dataset(
                filename,
                engine="cfgrib",
                filter_by_keys={"typeOfLevel": level, "stepType": step},
                backend_kwargs={"errors": "ignore"}
            )
        else:
            ds = xr.open_dataset(
                filename, 
                engine="cfgrib", 
                filter_by_keys={"typeOfLevel": level}, 
                backend_kwargs={"errors": "ignore"}
            )
        # print key references if the return dataset is empty. ask users to recheck the filtering keys
        if len(ds.data_vars) == 0:
            print("The filter keys may be incorrect. Please check the following reference:")
            for i in key_details:
                print(i)

        return ds
    
    # ask users to recheck the file parameters if there is a 404 HTTP error
    except HTTPError as err:
        if err.code == 404:
            print(f"{URL} does not exist. Please check the parameters again.")

`cycle_runtime`: the model cycle runtime (i.e. 00, 06, 12, 18)\
`forcast_hour` : the forecast hour of product from 000 - 384\
`year`, `month`, `day` : Azure container retains GFS data for 3 months\
`resolution_degree` : degree resolution of the data (i.e. 0.25, 0.5, 1.00)

In [ ]:
from datetime import datetime

# define variables to fetch yesterday's data for plotting purposes
yesterday = datetime.now().day - 1
current_month = datetime.now().month
current_year = datetime.now().year

URL = file_path(cycle_runtime=12, forecast_hour=102, year=current_year, month=current_month, day=yesterday, resolution_degree=.25)
ds_yesterday = read_into_xarray_dataset(URL, 'pressureFromGroundLayer')

print(URL)
ds_yesterday

#### Filter the dataset with US boundary, convert it into pandas dataframe, rename and reorder the columns.

In [ ]:
df_usa = (ds_yesterday[['t', 'r', 'q', 'u', 'v']]
          .sel(latitude = slice(50, 24), longitude = slice(235, 293))
          .to_dataframe()
          .sort_values('latitude')
          .reset_index()
          )

df_usa.rename(columns = {'t' : ds_yesterday['t'].standard_name, 
                         'r' : ds_yesterday['r'].standard_name, 
                         'q' : ds_yesterday['q'].standard_name, 
                         'u' : ds_yesterday['u'].standard_name, 
                         'v' : ds_yesterday['v'].standard_name}, inplace = True
                         )

df_usa.drop('valid_time', axis = 1, inplace = True)
df_usa = df_usa.iloc[:, [7,8,9,0,1,2,3,4,5,6]]

df_usa.head()

---

## Visualization

#### Loop through six different forecast hour periods to see global temperature changes with 1.00 degree resolution 

*NOTE: This process may take a while to run. Please be patient.*

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from alive_progress import alive_bar

# define a list of forecast hours 
forecast_hours = [x for x in range(12, 72 + 1, 12)]

fig = plt.figure(figsize = (15, 12))
rows = len(forecast_hours) // 2 + len(forecast_hours) % 2
cols = 2

# add alive bar to see the progrees for generating each subplots
with alive_bar(len(forecast_hours), force_tty=True, title='Running', length=20, bar = 'smooth') as bar:
    for n, forecast_hour in enumerate(forecast_hours):
        URL = file_path(12, forecast_hour, current_year, current_month, yesterday, 1.)
        ds = read_into_xarray_dataset(URL, 'pressureFromGroundLayer')
        ax = plt.subplot(rows, cols, n + 1, projection=ccrs.Robinson())
        ax.coastlines(resolution="10m")
        # convert temperature measurement from K to C
        (ds['t'] - 273.15).plot(ax = ax,
                                cmap=plt.cm.coolwarm,
                                transform=ccrs.PlateCarree(), 
                                cbar_kwargs={"shrink": 0.6, "label": "Temperature [C]"}
                                )
        plt.title(f"forecast {ds.step.values.astype('timedelta64[h]')}")  
        plt.grid(False)
        bar()

plt.suptitle("Three day forecast of Temperature from " 
             f"{ds.time.values.astype('datetime64[s]')} " 
             "with 1.00 degree resolution", 
             size = 18
             )

plt.subplots_adjust(top = 0.95)
plt.tight_layout()

#### Loop through all different resolution degrees to see the difference of picture clarity

*NOTE: This process may take a while to run. Please be patient.*

In [ ]:
fig = plt.figure(figsize=(15, 18))

resolution_degrees = [0.25, 0.5, 1.]

with alive_bar(len(resolution_degrees), force_tty=True, title='Running', length=20, bar = 'smooth') as bar:
    for n, resolution_degree in enumerate(resolution_degrees):
        URL = file_path(12, 24, current_year, current_month, yesterday, resolution_degree)
        ds = read_into_xarray_dataset(URL, 'pressureFromGroundLayer')
        ax = plt.subplot(3, 1, n + 1)
        ds['r'].plot(ax = ax)
        plt.title(f"forecast {ds.step.values.astype('timedelta64[h]')} " 
                  f"from {ds.time.values.astype('datetime64[s]')} " 
                  f"with {resolution_degree} degree resolution"
                  )
        plt.grid(False)  
        bar()

plt.tight_layout()

#### Focusing on the United States, add states and borders using `cartopy.feature` to make it more aesthetically appealing

In [ ]:
import cartopy.feature as cfeature

fig = plt.figure(figsize = (15, 10))
ds_usa = ds_yesterday.sel(latitude = slice(50, 20), longitude = slice(235, 294))
ax = plt.axes(projection=ccrs.PlateCarree())
(ds_usa['t'] - 273.15).plot.contourf(
                            ax = ax,
                            cmap = plt.cm.coolwarm,
                            transform = ccrs.PlateCarree(), 
                            cbar_kwargs={"shrink": 0.6, "label": "Temperature [C]"}
                            )
ax.coastlines()
ax.add_feature(cfeature.STATES.with_scale('10m'), zorder=3, linewidth=0.8, edgecolor='black')
ax.add_feature(cfeature.BORDERS, linewidth=0.8)

plt.title(f"forecast {ds_usa.step.values.astype('timedelta64[h]')} " 
          f"from {ds_usa.time.values.astype('datetime64[s]')} " 
         )

plt.show()

#### Focus on United States with 30 different meteorological variables  

*NOTE: This process may take a while to run. Please be patient.*

In [ ]:
URL = file_path(cycle_runtime=12, forecast_hour=12, year=current_year, month=current_month, day=yesterday, resolution_degree=0.25)
ds = read_into_xarray_dataset(URL, 'surface', 'instant')

# filter out unknown data variable
filtered_vars = [var for var in ds.data_vars if var != 'unknown']
ds_usa = ds.sel(latitude = slice(50, 20), longitude = slice(235, 294))

fig = plt.figure(figsize = (50, 80))
cols = 3
rows = len(filtered_vars) // cols + len(filtered_vars) % cols

with alive_bar(len(filtered_vars), force_tty=True, title='Running', length=20, bar = 'smooth') as bar:
    for n, var in enumerate(filtered_vars):
        ax = plt.subplot(rows, cols, n + 1, projection=ccrs.PlateCarree())
        ax.coastlines(resolution="10m")
        im = ds_usa[var].plot(ax = ax,
                        transform=ccrs.PlateCarree(),
                        add_colorbar=False 
                        )
        cb = plt.colorbar(im)
        cb.set_label(label=ds_usa[var].attrs['GRIB_units'], size=30)
        cb.ax.tick_params(labelsize=20)
        plt.title(f"{ds_usa[var].attrs['long_name']}", fontdict={'fontsize':40})  
        plt.grid(False)
        bar()

plt.tight_layout()

---

### Animation - Temperature forecasting to 96 hours

In [ ]:
import imageio

# create a folder `img` and store all the plots in it
try:
    os.mkdir('./img')
except:
    print('Directory already exists')

# define a function to generate temperature plot per forecast hour
def make_frame(forecast_hour:int):    
    plt.figure(figsize=(15, 12))
    
    URL = file_path(12, forecast_hour, current_year, current_month, yesterday, 1.)
    ds = read_into_xarray_dataset(URL, 'pressureFromGroundLayer')

    ax = plt.axes(projection=ccrs.Robinson())
    ax.coastlines(resolution="10m")
    im = (ds['t'] - 273.15).plot(ax = ax,
                            cmap=plt.cm.coolwarm,
                            transform=ccrs.PlateCarree(), 
                            add_colorbar=False
                            )
    # customize the colorbar
    cb = fig.colorbar(im, location = 'bottom', pad = .03)
    cb.set_label(label=f"{ds['t'].attrs['long_name']} [C]", size=15)
    cb.ax.tick_params(labelsize=12)

    plt.title(f"Forecast {ds.step.values.astype('timedelta64[h]')} from {ds.time.values.astype('datetime64[s]')}",  fontdict={'fontsize':18})  
    plt.grid(False)
    plt.tight_layout()

    plt.savefig(f'./img/temp_forecast_{forecast_hour}_hours.png', transparent=False, facecolor='white')
    plt.close()

*NOTE: This process may take a while to run.  It is the longest of the cells we are demonstrating. Please be patient.*

In [ ]:
# generate plots with given forecast hours and save them into .png file
forecast_hour = [x for x in range(3, 96 + 1, 3)]
with alive_bar(len(forecast_hour), force_tty=True, title='Running', length=20, bar = 'smooth') as bar:
    for t in forecast_hour:
        make_frame(t)
        bar()

# use `imageio` to create gif with given plots 
frames = []
for t in forecast_hour:
    image = imageio.v2.imread(f'./img/temp_forecast_{t}_hours.png')
    frames.append(image)

imageio.mimsave('./img/temp_forecast.gif',
                frames,
                loop = 65535,
                fps = 2)

### Animation - Relative Humidity forecasting to 96 hours

In [ ]:
# define a function to generate relative humidity plot per forecast hour
def make_frame(forecast_hour:int):
    plt.figure(figsize=(15, 12))
    
    URL = file_path(12, forecast_hour, current_year, current_month, yesterday, 1.)
    ds = read_into_xarray_dataset(URL, 'pressureFromGroundLayer')

    ax = plt.axes(projection=ccrs.Robinson())
    ax.coastlines(resolution="10m")

    im = ds['r'].plot(ax = ax,
                transform=ccrs.PlateCarree(), 
                add_colorbar=False
                )
    cb = fig.colorbar(im, location = 'bottom', pad = .03)
    cb.set_label(label=f"{ds['r'].attrs['long_name']} [{ds['r'].attrs['GRIB_units']}]", size=15)
    cb.ax.tick_params(labelsize=12)

    plt.title(f"Forecast {ds.step.values.astype('timedelta64[h]')} from {ds.time.values.astype('datetime64[s]')}", fontdict={'fontsize':18})  
    plt.grid(False)
    plt.tight_layout()

    plt.savefig(f'./img/relative_humidity_forecast_{forecast_hour}_hours.png', transparent=False, facecolor='white')
    plt.close()

In [ ]:
# generate plots with given forecast hours and save them into .png file
forecast_hour = [x for x in range(3, 96 + 1, 3)]
with alive_bar(len(forecast_hour), force_tty=True, title='Running', length=20, bar = 'smooth') as bar:
    for t in forecast_hour:
        make_frame(t)
        bar()

# use `imageio` to create gif with given plots 
frames = []
for t in forecast_hour:
    image = imageio.v2.imread(f'./img/relative_humidity_forecast_{t}_hours.png')
    frames.append(image)

imageio.mimsave('./img/relative_humidity_forecast.gif',
                frames,
                loop = 65535,
                fps = 2)